## Objectives


This lab contains the following tasks:

Mark all launch sites on a map
Mark the success/failed launches for each site on the map
Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [25]:
!pip install folium==0.7.0
!pip3 install wget

In [26]:
import folium
import wget
import pandas as pd

In [27]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [28]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file)

100% [............................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [29]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


NameError: name 'df' is not defined

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [30]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [31]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [32]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [33]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


In [34]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [35]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
def marked_map(mp):
    site_map = mp
    marker_cluster = MarkerCluster().add_to(site_map)
    for index, record in spacex_df.iterrows():
        # TODO: Create and add a Marker cluster to the site map
        label = record["Launch Site"]
        folium.Marker(
        location=[record.Lat, record.Long],
        popup=label,
        icon=folium.Icon(color="green", icon="ok-sign"),
    ).add_to(marker_cluster)

        circle = folium.Circle(
            [record.Lat, record.Long],
            radius=50,
            color="#ff0000	",
            fill_color='#ff7f7f',
            fill=True).add_child(folium.Popup(label))

        marker = folium.map.Marker(
            [record.Lat, record.Long],
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label)
        )


        site_map.add_child(circle)
        site_map.add_child(marker)

    return site_map

In [36]:
marked_map(site_map)

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


 Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [37]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [38]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [39]:
launch_sites_df.iloc[0, 1]

28.56230197

In [40]:

def closest_long_lat_dist(lat_list, long_list, lat2, lon2):
    closest_lat = 0
    closest_long = 0
    closest_dist = 0
    for lat, long in zip(lat_list, long_list):
        calc = calculate_distance(lat, long, lat2, lon2)

        if not closest_dist:
            closest_dist = calculate_distance(lat, long, lat2, lon2)
            closest_lat = lat
            closest_long = long
        elif closest_dist < calc:
            continue
        closest_dist = calc
        closest_lat = lat
        closest_long = long
        
    return closest_lat, closest_long, closest_dist

closest_lat1, closest_long1, closest_dist1 = closest_long_lat_dist(launch_sites_df.Lat, launch_sites_df.Long, 28.56336, -80.5679)

In [41]:
closest_lat1


28.56319718

In [42]:
closest_long1

-80.57682003

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [43]:
def mark(the_map, lat, long, label):
    distance_marker = folium.Marker(
       [lat, long],
       icon=DivIcon(
           icon_size=(20,20),
           icon_anchor=(0,0),
           html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(label),
           )
       )
    the_map.add_child(distance_marker)
    
mark(site_map, 28.56343, -80.5679, closest_dist1)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [44]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines = folium.PolyLine(locations=[[closest_lat1, closest_long1],
                                    [28.56336, -80.5679]
                                   ]
                      , weight=1)
site_map.add_child(lines)

In [45]:
highway_lat = 28.56274
highway_long = -80.5707
closest_lat2, closest_long2, closest_dist2 = closest_long_lat_dist(launch_sites_df.Lat, launch_sites_df.Long, highway_lat, highway_long)
mark(site_map, highway_lat, highway_long, closest_dist2)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
line2 = folium.PolyLine(locations=[[closest_lat2, closest_long2],
                                    [highway_lat, highway_long]
                                   ]
                      , weight=1)
site_map.add_child(line2)


In [46]:
railway_lat = 28.57207
railway_long = -80.585272
closest_lat3, closest_long3, closest_dist3 = closest_long_lat_dist(launch_sites_df.Lat, launch_sites_df.Long, railway_lat, railway_long)
mark(site_map, railway_lat, railway_long, closest_dist3)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
line3 = folium.PolyLine(locations=[[closest_lat3, closest_long3],
                                    [railway_lat, railway_long]
                                   ]
                      , weight=1)
site_map.add_child(line3)

In [47]:
city_lat = 28.4065
city_long = -80.59854
closest_lat4, closest_long4, closest_dist4 = closest_long_lat_dist(launch_sites_df.Lat, launch_sites_df.Long, city_lat, city_long)
mark(site_map, city_lat, city_long, closest_dist4)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
line4 = folium.PolyLine(locations=[[closest_lat4, closest_long4],
                                    [city_lat, city_long]
                                   ]
                      , weight=1)
site_map.add_child(line4)

- The closest launch site to coast, highway and raiway is CCAFS SLC-40

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


After plot distance lines to the proximities, we can say the following:

* launch sites Are in close proximity to railways
* launch sites Are in close proximity to highways
* launch sites Are in close proximity to coastline
* launch sites keep certain distance away from cities
